In [2]:
# Importing essential libraries
import pandas as pd
import numpy as np

In [3]:
# Loading the 'movie_metadata.tsv' dataset
movie_metadata = pd.read_csv('movie_metadata.tsv', sep='\t', header=None)

In [4]:
# Loading the 'plot_summaries.tsv' dataset
plot_summary = pd.read_csv('plot_summaries.tsv', sep='\t', header=None)

In [5]:
movie_metadata.head()

,0,1,2,3,4,5,6,7,8
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [6]:
movie_metadata.shape

(81741, 9)

In [7]:
plot_summary.head()

,0,1
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [8]:
plot_summary.shape

(42303, 2)

In [9]:
# Renaming the required columns
movie_metadata.columns = ["movie_id", 1, "movie_name", 3, 4, 5, 6, 7, "genre"]
plot_summary.columns = ["movie_id", "plot"]

In [10]:
# Merging both dataframes
df = pd.merge(movie_metadata[['movie_id', 'movie_name', 'genre']], plot_summary, on='movie_id')

In [11]:
df.head()

,movie_id,movie_name,genre,plot
0,975900,Ghosts of Mars,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","Set in the second half of the 22nd century, th..."
1,9363483,White Of The Eye,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...
2,261236,A Woman in Flames,"{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra..."
3,18998739,The Sorcerer's Apprentice,"{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","Every hundred years, the evil Morgana returns..."
4,6631279,Little city,"{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","Adam, a San Francisco-based artist who works a..."


In [12]:
df.shape

(42204, 4)

In [13]:
# Cleaning the genre column
df['genre'][0]

'{"/m/01jfsb": "Thriller", "/m/06n90": "Science Fiction", "/m/03npn": "Horror", "/m/03k9fj": "Adventure", "/m/0fdjb": "Supernatural", "/m/02kdv5l": "Action", "/m/09zvmj": "Space western"}'

In [14]:
import json
json.loads(df['genre'][0]).values()

dict_values(['Thriller', 'Science Fiction', 'Horror', 'Adventure', 'Supernatural', 'Action', 'Space western'])

In [15]:
df['genre'] = df['genre'].apply(lambda x: list(json.loads(x).values()))
df.head()

,movie_id,movie_name,genre,plot
0,975900,Ghosts of Mars,"[Thriller, Science Fiction, Horror, Adventure,...","Set in the second half of the 22nd century, th..."
1,9363483,White Of The Eye,"[Thriller, Erotic thriller, Psychological thri...",A series of murders of rich young women throug...
2,261236,A Woman in Flames,[Drama],"Eva, an upper class housewife, becomes frustra..."
3,18998739,The Sorcerer's Apprentice,"[Family Film, Fantasy, Adventure, World cinema]","Every hundred years, the evil Morgana returns..."
4,6631279,Little city,"[Romantic comedy, Ensemble Film, Comedy-drama,...","Adam, a San Francisco-based artist who works a..."


In [16]:
# Remove rows with 0 genre tags
df = df[df['genre'].apply(lambda x: False if len(x)==0 else True)]
print("After removing the rows: {}".format(df.shape))

After removing the rows: (41793, 4)


In [17]:
all_genres = sum(df['genre'] ,[])
len(set(all_genres))

363

In [18]:
# Importing essential libraries
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
def plot_clean(text):
  text = re.sub(pattern='[^a-zA-Z]', repl=' ', string=text)
  text = text.lower()
  text = ' '.join(text.split())
  return text

In [20]:
df['clean_plot'] = df['plot'].apply(lambda x: plot_clean(x))
df[['plot', 'clean_plot']].head()

,plot,clean_plot
0,"Set in the second half of the 22nd century, th...",set in the second half of the nd century the f...
1,A series of murders of rich young women throug...,a series of murders of rich young women throug...
2,"Eva, an upper class housewife, becomes frustra...",eva an upper class housewife becomes frustrate...
3,"Every hundred years, the evil Morgana returns...",every hundred years the evil morgana returns t...
4,"Adam, a San Francisco-based artist who works a...",adam a san francisco based artist who works as...


In [21]:
# Removing the stopwords from 'clean_plot'
stopwords_set = set(stopwords.words('english'))

def plot_stopwords(text):
  words = text.split()
  words = [word for word in words if not word in stopwords_set]
  text = ' '.join(words)
  return text

In [22]:
df['clean_noStopword_plot'] = df['clean_plot'].apply(lambda x: plot_stopwords(x))
df[['clean_plot', 'clean_noStopword_plot']].head()

,clean_plot,clean_noStopword_plot
0,set in the second half of the nd century the f...,set second half nd century film depicts mars p...
1,a series of murders of rich young women throug...,series murders rich young women throughout ari...
2,eva an upper class housewife becomes frustrate...,eva upper class housewife becomes frustrated l...
3,every hundred years the evil morgana returns t...,every hundred years evil morgana returns claim...
4,adam a san francisco based artist who works as...,adam san francisco based artist works cab driv...


In [23]:
# Stemming the words from 'clean_noStopwords_plot'
ps = PorterStemmer()

def plot_stemming(text):
  words = text.split()
  words = [ps.stem(word) for word in words]
  text = ' '.join(words)
  return text

In [24]:
df['clean_noStopword_stem_plot'] = df['clean_noStopword_plot'].apply(lambda x: plot_stemming(x))
df[['clean_noStopword_plot', 'clean_noStopword_stem_plot']].head()

,clean_noStopword_plot,clean_noStopword_stem_plot
0,set second half nd century film depicts mars p...,set second half nd centuri film depict mar pla...
1,series murders rich young women throughout ari...,seri murder rich young women throughout arizon...
2,eva upper class housewife becomes frustrated l...,eva upper class housewif becom frustrat leav a...
3,every hundred years evil morgana returns claim...,everi hundr year evil morgana return claim fin...
4,adam san francisco based artist works cab driv...,adam san francisco base artist work cab driver...


In [25]:
# Convert Genre into Target variables using MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['genre'])

In [26]:
X = df['clean_noStopword_stem_plot']

In [27]:
# Splitting the dataset into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [28]:
# Extracting independent features using TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_df=0.8, max_features=10000)
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [29]:
# Importing essential libraries
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

In [30]:
lr = LogisticRegression()
ovr_clf = OneVsRestClassifier(lr)

In [31]:
# Fit model on train data
ovr_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 48 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 243 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


OneVsRestClassifier(estimator=LogisticRegression())

In [32]:
# Predicting values for test set
y_pred_ovr = ovr_clf.predict(X_test)
y_pred_ovr[5]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [33]:
mlb.inverse_transform(y_pred_ovr)[5]

('Drama',)

In [34]:
# Model Evaluation
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred_ovr, average="micro"))

0.324498931732628


In [35]:
# Changing the threshold probability from 0.5 to 0.3
y_pred_prob = ovr_clf.predict_proba(X_test)
y_pred_new = (y_pred_prob >= 0.2).astype(int)

In [36]:
f1_score(y_test, y_pred_new, average="micro")

0.47357308040352586

In [37]:
# Creating a function
def predict_genre_tags(text):
    text = plot_clean(text)
    text = plot_stopwords(text)
    text = plot_stemming(text)
    text_vec = tfidf.transform([text])
    text_pred = ovr_clf.predict(text_vec)
    return mlb.inverse_transform(text_pred)

In [38]:
# Prediction 1
movie_name = 'Titanic (1998)'
plot_summary = "After winning a trip on the RMS Titanic during a dockside card game, American Jack Dawson spots the society girl Rose DeWitt Bukater who is on her way to Philadelphia to marry her rich snob fiancé Caledon Hockley. Rose feels helplessly trapped by her situation and makes her way to the aft deck and thinks of suicide until she is rescued by Jack. Cal is therefore obliged to invite Jack to dine at their first-class table where he suffers through the slights of his snobbish hosts. In return, he spirits Rose off to third-class for an evening of dancing, giving her the time of her life. Deciding to forsake her intended future all together, Rose asks Jack, who has made his living making sketches on the streets of Paris, to draw her in the nude wearing the invaluable blue diamond Cal has given her. Cal finds out and has Jack locked away. Soon afterwards, the ship hits an iceberg and Rose must find Jack while both must run from Cal even as the ship sinks deeper into the freezing water."
actual_genre = ['Drama', 'Romance']
predicted_genre = predict_genre_tags(plot_summary)
print('Movie: {}\nPredicted genres: {}\nActual genres: {}'.format(movie_name, predicted_genre[0], actual_genre))

Movie: Titanic (1998)
Predicted genres: ('Drama',)
Actual genres: ['Drama', 'Romance']


In [46]:
# Prediction 2
movie_name = 'Avatar (2009)'
plot_summary = "On the lush alien world of Pandora live the Na'vi, beings who appear primitive but are highly evolved. Because the planet's environment is poisonous, human/Na'vi hybrids, called Avatars, must link to human minds to allow for free movement on Pandora. Jake Sully (Sam Worthington), a paralyzed former Marine, becomes mobile again through one such Avatar and falls in love with a Na'vi woman (Zoe Saldana). As a bond with her grows, he is drawn into a adventurous battle for the survival of her world."
actual_genre = ['Action', 'Adventure', 'Fantasy']
predicted_genre = predict_genre_tags(plot_summary)
print('Movie: {}\nPredicted genres: {}\nActual genres: {}'.format(movie_name, predicted_genre[0], actual_genre))

Movie: Avatar (2009)
Predicted genres: ()
Actual genres: ['Action', 'Adventure', 'Fantasy']


In [40]:
# Prediction 3
movie_name = 'Conjuring (2013)'
plot_summary = "In 1971, Carolyn and Roger Perron move their family into a dilapidated Rhode Island farm house and soon strange things start happening around it with escalating nightmarish terror. In desperation, Carolyn contacts the noted paranormal investigators, Ed and Lorraine Warren, to examine the house. What the Warrens discover is a whole area steeped in a satanic haunting that is now targeting the Perron family wherever they go. To stop this evil, the Warrens will have to call upon all their skills and spiritual strength to defeat this spectral menace at its source that threatens to destroy everyone involved."
actual_genre = ['Horror', 'Thriller']
predicted_genre = predict_genre_tags(plot_summary)
print('Movie: {}\nPredicted genres: {}\nActual genres: {}'.format(movie_name, predicted_genre[0], actual_genre))

Movie: Conjuring (2013)
Predicted genres: ('Horror',)
Actual genres: ['Horror', 'Thriller']


In [41]:
# Prediction 4
movie_name = 'The Hangover (2009)'
plot_summary = "Three buddies wake up from a bachelor party in Las Vegas, with no memory of the previous night and the bachelor missing. They make their way around the city in order to find their friend before his wedding."
actual_genre = ['Comedy']
predicted_genre = predict_genre_tags(plot_summary)
print('Movie: {}\nPredicted genres: {}\nActual genres: {}'.format(movie_name, predicted_genre[0], actual_genre))

Movie: The Hangover (2009)
Predicted genres: ('Comedy', 'Romance Film')
Actual genres: ['Comedy']


In [42]:
# Prediction 5
movie_name = 'La La Land (2016)'
plot_summary = "The story of aspiring actress Mia and dedicated jazz musician Sebastian, who struggle to make ends meet while pursuing their dreams in a city known for destroying hopes and breaking hearts. With modern-day Los Angeles as the backdrop, this musical about everyday life explores what more important: a once-in-a-lifetime love or the spotlight."
actual_genre = ['Comedy', 'Drama', 'Music']
predicted_genre = predict_genre_tags(plot_summary)
print('Movie: {}\nPredicted genres: {}\nActual genres: {}'.format(movie_name, predicted_genre[0], actual_genre))

Movie: La La Land (2016)
Predicted genres: ('Drama',)
Actual genres: ['Comedy', 'Drama', 'Music']
